**Task 09: Data linking**

In [10]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4/"

In [11]:
from rdflib import Graph, Namespace, Literal, URIRef
g1 = Graph()
g2 = Graph()
g3 = Graph()
g1.parse(github_storage+"resources/data03.rdf", format="xml")
g2.parse(github_storage+"resources/data04.rdf", format="xml")

<Graph identifier=Nec4a5f9482584c8eb214589372993409 (<class 'rdflib.graph.Graph'>)>

Search for individuals in the two graphs and link them using the OWL: sameAs property, insert these matches in g3. We consider that two individuals are the same if they have the same nickname and family name. Keep in mind that the URIs do not have to be the same for the same individual in the two graphs.

In [24]:
from rdflib.plugins.sparql import prepareQuery
from rdflib.namespace import RDF, RDFS, OWL


vcard = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")
ns3 = Namespace("http://data.three.org#") ##Follow the Documento
ns4 = Namespace("http://data.four.org#") ##Follow the Documento

#Search for individuals in the two graphs
for s,p,o in g1:
  print(s,p,o)

print("-------------------------------------------------------------------------")

for s,p,o in g2:
  print(s,p,o)

http://www.w3.org/2001/vcard-rdf/3.0/Given http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://data.three.org#JohnDoe http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.three.org#Person
http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Property
http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://data.three.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#Family Smith
http://data.three.org#SaraJones http://www.w3.org/2001/vcard-rdf/3.0#Given Sara
http://data.three.org#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.three.org#Person
http://data.three.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#FN John Smith
http://data.three.org#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.three.org#Person
http://www.w3.org/2001/vcard-rdf

In [27]:
#Search for nicknames(Consider it`s like given name) and family names in the two graphs = FullNAME
q1 = prepareQuery('''
  SELECT 
    ?Subject ?GIVEN ?FAMILY ?FN
  WHERE { 
    ?Subject rdf:type ns:Person.
    ?Subject vcard:FN ?FN.
    
    OPTIONAL{?Subject vcard:Family ?GIVEN.
    ?Subject vcard:Given ?FAMILY.
    }
    
  }
  ''',
  initNs = {"rdf":RDF, "ns":ns3, "vcard": vcard}
)
g_aux1 = []
for r in g1.query(q1):
  g_aux1.append(r)#insert "append"
  print(r)



print("-------------------------------------------------------------------------")
q2 = prepareQuery('''
  SELECT 
    ?Subject ?GIVEN ?FAMILY ?FN
  WHERE { 
    ?Subject rdf:type ns:Person.
    ?Subject vcard:FN ?FN.
    
    OPTIONAL{?Subject vcard:Family ?GIVEN.
    ?Subject vcard:Given ?FAMILY.
    }
    
  }
  ''',
  initNs = {"rdf":RDF, "ns":ns4, "vcard": vcard}
)
g_aux2 = []
for r in g2.query(q2):
  g_aux2.append(r)
  print(r)



(rdflib.term.URIRef('http://data.three.org#HarryPotter'), None, None, rdflib.term.Literal('Harry Potter', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://data.three.org#JohnSmith'), rdflib.term.Literal('Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('John', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('John Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://data.three.org#JohnDoe'), rdflib.term.Literal('Doe', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('John', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('John Doe', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://data.three.org#SaraJones'), rdflib.term.Literal('Jones', datatype=r

In [29]:
#sameAs
for s1 in g_aux1:
  for s2 in g_aux2:
    if(s1.FN == s2.FN or (s1.GIVEN==s2.GIVEN and s1.FAMILY == s2.FAMILY)):
      result=s1.Subject, OWL.sameAs, s2.Subject
      g3.add(result)

for s,p,o in g3:
  print(s,p,o)

http://data.three.org#SaraJones http://www.w3.org/2002/07/owl#sameAs http://data.four.org#0001
http://data.three.org#HarryPotter http://www.w3.org/2002/07/owl#sameAs http://data.four.org#0003
http://data.three.org#JohnDoe http://www.w3.org/2002/07/owl#sameAs http://data.four.org#0005
http://data.three.org#JohnSmith http://www.w3.org/2002/07/owl#sameAs http://data.four.org#0002
